In [29]:
import plotly.plotly as py
import plotly.graph_objs as go
from datascience import *
import numpy as np
import pandas as pd

py.sign_in('oikobill', 'nebugeZ772mlIhUIxTPh')

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/merged_election_data.csv").drop("Unnamed: 0", axis=1)
df.head(5)

,County Name,State,ObamaVotes.12,RomneyVotes.12,votes_dem.16,votes_gop.16,total_votes.16,per_dem.16,per_gop.16,diff.16,...,public_transportation_to_work,working_at_home,occupation_management_business_science_arts,income_less_than_10000,income_10000_to_24999,income_25000_to_49999,income_50000_to_99999,income_100000_to_149999,income_150000_to_199999,income_more_than_200000
0,2baltimore,maryland,201042.0,25501.0,178562.0,22726.0,208980.0,0.854445,0.108747,491.0,...,49262.0,7150.0,98932.0,31068.0,47053.0,63771.0,64532.0,19217.0,19217.0,6272.0
1,2bedford,virginia,1225.0,1527.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,116.0,861.0,383.0,661.0,850.0,585.0,217.0,217.0,9.0
2,2fairfax,virginia,6636.0,4762.0,7363.0,3695.0,11854.0,0.621141,0.311709,1513.0,...,1037.0,541.0,6349.0,176.0,541.0,1057.0,2661.0,2085.0,2085.0,1074.0
3,2franklin,virginia,2830.0,1496.0,2516.0,1421.0,4040.0,0.622772,0.351733,104.0,...,0.0,38.0,945.0,595.0,804.0,884.0,861.0,289.0,289.0,31.0
4,2richmond,virginia,71037.0,19732.0,74640.0,14254.0,94724.0,0.787973,0.150479,2118.0,...,6534.0,3146.0,35339.0,10792.0,18415.0,22262.0,20683.0,5977.0,5977.0,2951.0


In [6]:
list(df.columns.values)

['County Name',
 'State',
 'ObamaVotes.12',
 'RomneyVotes.12',
 'votes_dem.16',
 'votes_gop.16',
 'total_votes.16',
 'per_dem.16',
 'per_gop.16',
 'diff.16',
 'per_point_diff.16',
 'bushVote.04',
 'kerryVote.04',
 'Total.Precincts.08',
 'Precincts.Reporting.08',
 'Obama.08',
 'McCain.08',
 'Other.08',
 'name',
 'x_coord',
 'y_coord',
 'state_abbr',
 'total_population',
 'white_population',
 'white_population_error',
 'black_population',
 'black_population_error',
 'total_households',
 'average_household_size',
 'average_family_size',
 'fertility',
 'enrolled_hs',
 'enrolled_higher_ed',
 'edu_less_than_9th',
 'edu_hs_no_diploma',
 'edu_hs_diploma',
 'edu_college_no_degree',
 'edu_college_bachelors',
 'edu_college_graduate_professional',
 'in_labor_force_yes',
 'in_labor_force_no',
 'public_transportation_to_work',
 'working_at_home',
 'occupation_management_business_science_arts',
 'income_less_than_10000',
 'income_10000_to_24999',
 'income_25000_to_49999',
 'income_50000_to_99999',
 '

In [148]:
# df_filtered_2016 = df[['votes_dem.16','votes_gop.16', "state_abbr"]]
# new_df_2016 = df_filtered_2016.groupby('state_abbr').sum()
# new_df_2016['code'] = new_df_2016.axes[0]

In [3]:
df[df.State == "florida"]

,County Name,State,ObamaVotes.12,RomneyVotes.12,votes_dem.16,votes_gop.16,total_votes.16,per_dem.16,per_gop.16,diff.16,...,public_transportation_to_work,working_at_home,occupation_management_business_science_arts,income_less_than_10000,income_10000_to_24999,income_25000_to_49999,income_50000_to_99999,income_100000_to_149999,income_150000_to_199999,income_more_than_200000
30,alachua,florida,69634.0,48765.0,75370.0,46584.0,127827.0,0.589625,0.364430,1214.0,...,3898.0,3969.0,52095.0,13374.0,19642.0,24419.0,24085.0,9382.0,9382.0,3436.0
121,baker,florida,2310.0,8974.0,2112.0,10294.0,12634.0,0.167168,0.814785,2506.0,...,66.0,161.0,2641.0,556.0,1309.0,2553.0,2892.0,732.0,732.0,202.0
154,bay,florida,22037.0,56863.0,21689.0,62010.0,87151.0,0.248867,0.711524,1625.0,...,423.0,1772.0,23857.0,4141.0,11523.0,20141.0,22789.0,6976.0,6976.0,1406.0
259,bradford,florida,3324.0,8216.0,2924.0,8913.0,12098.0,0.241693,0.736733,2102.0,...,0.0,163.0,2585.0,909.0,1759.0,2681.0,2856.0,431.0,431.0,142.0
271,brevard,florida,122972.0,159270.0,119525.0,181620.0,314337.0,0.380245,0.577788,2154.0,...,925.0,9792.0,87313.0,13333.0,37234.0,60816.0,70312.0,25012.0,25012.0,6637.0
285,broward,florida,507430.0,243732.0,546956.0,258521.0,822837.0,0.664720,0.314183,1218.0,...,22621.0,35719.0,295892.0,44795.0,108474.0,170896.0,202060.0,82030.0,82030.0,29311.0
351,calhoun,florida,1664.0,4366.0,1236.0,4647.0,6067.0,0.203725,0.765947,1429.0,...,0.0,109.0,1330.0,497.0,1337.0,1556.0,957.0,302.0,302.0,62.0
455,charlotte,florida,35896.0,47988.0,33421.0,60196.0,96323.0,0.346968,0.624939,1122.0,...,185.0,2995.0,16422.0,4544.0,12978.0,22582.0,22836.0,5736.0,5736.0,1527.0
513,citrus,florida,28453.0,44657.0,22765.0,54377.0,79590.0,0.286028,0.683214,1343.0,...,178.0,2160.0,12755.0,4635.0,13613.0,19747.0,16311.0,3954.0,3954.0,932.0
542,clay,florida,25745.0,69996.0,27768.0,74898.0,106363.0,0.261068,0.704173,1813.0,...,263.0,3119.0,29509.0,2747.0,6908.0,16731.0,25745.0,10866.0,10866.0,1992.0


In [150]:
# new_df_2016['perc_diff'] = (new_df_2016['votes_dem.16']-new_df_2016['votes_gop.16'])/(new_df_2016['votes_dem.16']+new_df_2016['votes_gop.16'])
# new_df_2016.head()

,votes_dem.16,votes_gop.16,code,perc_diff
state_abbr,,,,
AK,93003.0,130413.0,AK,-0.167445
AL,718084.0,1306925.0,AL,-0.290784
AR,378729.0,677904.0,AR,-0.283140
AZ,936250.0,1021154.0,AZ,-0.043376
CA,5931283.0,3184721.0,CA,0.301290


In [151]:
# total_2016 = new_df_2016['votes_dem.16']+new_df_2016['votes_gop.16']
# new_df_2016['text'] = new_df_2016['code']+'<br>'+ "Democrats: " + (new_df_2016['votes_dem.16']*100/total_2016).astype('str')+"%"+'<br>'+ "Republicans: " + (new_df_2016['votes_gop.16']*100/total_2016).astype('str') + "%"

In [4]:
new_df_2016 = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2016.csv")

In [5]:
new_df_2016 = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2016.csv")


for col in new_df_2016.columns:
    new_df_2016[col] = new_df_2016[col].astype(str)

scl = [[0, 'rgb(255, 0, 0)'],[0.25, 'rgb(255, 122, 122)'],[0.4, "rgb(247, 133, 133)"],[0.45, 'rgb(255, 153, 153)'],
       [0.5, 'rgb(255, 255, 255)'],[0.55, 'rgb(160, 160, 255)'],[0.6, 'rgb(133, 177, 247)'], [0.75, 'rgb(122,122,255)'],[1, 'rgb(0, 0, 255)']]


data = [ dict(
        type='choropleth',
        #colorscale = scl,
        autocolorscale = False,
        locations = new_df_2016['code'],
        z = new_df_2016['perc_diff'].astype(float),
        locationmode = 'USA-states',
        text = new_df_2016['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage Difference"),
            colorscale = scl,
            zmin = -0.5,
            zmax = 0.5
        ) ]

layout = dict(
        title = '2016 Election',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [153]:
# df_filtered_2012 = df[['ObamaVotes.12','RomneyVotes.12', "state_abbr"]]
# new_df_2012 = df_filtered_2012.groupby('state_abbr').sum()
# new_df_2012['code'] = new_df_2012.axes[0]

In [160]:
# new_df_2012['perc_diff'] = (new_df_2012['ObamaVotes.12']-new_df_2012['RomneyVotes.12'])/(new_df_2012['ObamaVotes.12']+new_df_2012['RomneyVotes.12'])
# new_df_2012.head()

,ObamaVotes.12,RomneyVotes.12,code,perc_diff
state_abbr,,,,
AK,NaN,NaN,AK,NaN
AL,793620.0,1252453.0,AL,-0.224251
AR,389699.0,638467.0,AR,-0.241953
AZ,841607.0,1038907.0,AZ,-0.104918
CA,5967520.0,3890967.0,CA,0.210636


In [162]:
# total_2012 = new_df_2012['ObamaVotes.12']+new_df_2012['RomneyVotes.12']
# new_df_2012['text'] = new_df_2012['code']+'<br>'+ "Democrats: " + (new_df_2012['ObamaVotes.12']*100/total_2012).astype('str')+"%"+'<br>'+ "Republicans: " + (new_df_2012['RomneyVotes.12']*100/total_2012).astype('str') + "%"

In [6]:
new_df_2012 = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2012.csv")

for col in new_df_2012.columns:
    new_df_2012[col] = new_df_2012[col].astype(str)

scl = [[0, 'rgb(255, 0, 0)'],[0.25, 'rgb(255, 122, 122)'],[0.4, "rgb(247, 133, 133)"],[0.45, 'rgb(255, 153, 153)'],
       [0.5, 'rgb(255, 255, 255)'],[0.55, 'rgb(160, 160, 255)'],[0.6, 'rgb(133, 177, 247)'], [0.75, 'rgb(122,122,255)'],[1, 'rgb(0, 0, 255)']]


data = [ dict(
        type='choropleth',
        #colorscale = scl,
        autocolorscale = False,
        locations = new_df_2012['code'],
        z = new_df_2012['perc_diff'].astype(float),
        locationmode = 'USA-states',
        text = new_df_2012['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage Difference"),
            colorscale = scl,
            zmin = -0.5,
            zmax = 0.5
        ) ]

layout = dict(
        title = '2012 Election',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [12]:
#Finding the states that voted most differently
dems_2016_perc_diff = np.array(new_df_2016['perc_diff']).astype(float)
dems_2012_perc_diff = np.array(new_df_2012['perc_diff']).astype(float)
states = np.array(new_df_2016['state_abbr'])
assert np.all(states==np.array(new_df_2012['state_abbr']))

In [29]:
df = pd.DataFrame({'dems_2016':dems_2016_perc_diff, "dems_2012":dems_2012_perc_diff, "states":states})
df['diff'] = df['dems_2016']-df['dems_2012']
df = df.dropna()
df = df.sort_values("diff", ascending=False)
df.head()

,dems_2012,dems_2016,states,diff
44,-0.490767,-0.251895,UT,0.238872
4,0.210636,0.301290,CA,0.090654
43,-0.160678,-0.095185,TX,0.065493
3,-0.104918,-0.043376,AZ,0.061542
7,0.855325,0.914981,DC,0.059656


In [39]:
#how many states voted differently in the two elections?
dems_2012 = np.array(df['dems_2012'])
dems_2016 =  np.array(df['dems_2016'])
states_changed = np.count_nonzero(dems_2012*dems_2016<0)
states_unchanged = len(df)-states_changed

In [40]:
#most improved changed
data = [go.Bar(
            x=['voted differently', 'voted same'],
            y=[states_changed, states_unchanged]
    )]

py.iplot(data, filename='basic-bar')

In [33]:
#Of the states where most improvement or deterioration occured how many did democrats converted
most_impoved_states_dems = np.array(df['states'][:10])
most_worsened_states = np.array(df['states'][-10:])

In [41]:
#Of their most improved states how many did they win?

,dems_2012,dems_2016,states,diff
44,-0.490767,-0.251895,UT,0.238872
4,0.210636,0.301290,CA,0.090654
43,-0.160678,-0.095185,TX,0.065493
3,-0.104918,-0.043376,AZ,0.061542
7,0.855325,0.914981,DC,0.059656
19,0.234964,0.289287,MA,0.054323
47,0.143626,0.186990,WA,0.043363
10,-0.080496,-0.059224,GA,0.021273
45,0.031368,0.050902,VA,0.019533
20,0.255982,0.263242,MD,0.007261


County Demo

In [35]:
import urllib.request
import requests
import ast
from html.parser import HTMLParser
import string
import json

url = 'http://catalog.civicdashboards.com/dataset/cda82e8b-7a8b-411e-95ba-1200b921c35d/resource/5c5d19a0-b817-49e6-b76e-ea63a8e2c0f6/download/fd880c1e4d23463ca869f1122109b3eftemp.geojson'
florida_data = requests.get(url)
florida_data = florida_data.json()

#repub_democ_counties_url = 'http://dos.myflorida.com/elections/data-statistics/voter-registration-statistics/voter-registration-monthly-reports/voter-registration-current-by-county/'
#repub_democ_counties = requests.get(repub_democ_counties_url)

In [36]:
county_names = []
county_names_dict = {}

for county in florida_data['features']:
    for m in range(len(county['properties']['name'])):
        if county['properties']['name'][m:m+6] == 'County':
            county_names.append(county['properties']['name'][0:m-1].lower())
            county_names_dict[county['properties']['name'][0:m-1]] = county['properties']['name']
            
print(county_names)

['charlotte', 'seminole', 'baker', 'desoto', 'levy', 'alachua', 'pasco', 'hendry', 'okeechobee', 'broward', 'st. johns', 'gulf', 'glades', 'marion', 'duval', 'madison', 'osceola', 'lee', 'volusia', 'sarasota', 'indian river', 'clay', 'putnam', 'wakulla', 'holmes', 'escambia', 'flagler', 'union', 'brevard', 'suwannee', 'orange', 'martin', 'nassau', 'jefferson', 'santa rosa', 'hamilton', 'calhoun', 'hernando', 'miami-dade', 'pinellas', 'palm beach', 'hillsborough', 'collier', 'gilchrist', 'dixie', 'bay', 'gadsden', 'okaloosa', 'citrus', 'lafayette', 'manatee', 'monroe', 'columbia', 'sumter', 'washington', 'st. lucie', 'polk', 'taylor', 'leon', 'lake', 'highlands', 'hardee', 'bradford', 'liberty', 'franklin', 'walton', 'jackson']


In [37]:
red_counties_2012 = []
blue_counties_2012 = []
df = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/merged_election_data.csv").drop("Unnamed: 0", axis=1)
fl_voting_data = df[df.State == "florida"]

In [38]:
import re

fl_voting_data.columns = [c.replace(' ', '_') for c in fl_voting_data.columns]

for k, county in enumerate(county_names):
    county_cleaned = re.sub("\.", "", county)
    county_cleaned = re.sub(" ", "", county_cleaned)
    county_cleaned = re.sub(".*-", "", county_cleaned)
    row = fl_voting_data[fl_voting_data.County_Name == county_cleaned]
    if (len(row.index) != 1):
        print("Error: " + county_cleaned)
    else:
        if (row["ObamaVotes.12"].item() > row["RomneyVotes.12"].item()):
            blue_counties_2012.append(florida_data['features'][k])
        else:
            red_counties_2012.append(florida_data['features'][k])

    

In [39]:
red_data_2012 = {"type": "FeatureCollection"}
red_data_2012['features'] = red_counties_2012

blue_data_2012 = {"type": "FeatureCollection"}
blue_data_2012['features'] = blue_counties_2012

with open('florida-red-data-2012.json', 'w') as f:
    f.write(json.dumps(red_data_2012))
with open('florida-blue-data-2012.json', 'w') as f:
    f.write(json.dumps(blue_data_2012))


In [40]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = 'pk.eyJ1IjoiYWRhbWt1bGlkamlhbiIsImEiOiJjaXNya29kemwwNDNoMnRtMXBobGtvbWE4In0._k_JKvR8MknvKmiUsJ6C7g'

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
    )
])
layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/adnanhemani/data8-assets/master/florida-red-data-2012.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/adnanhemani/data8-assets/master/florida-blue-data-2012.json',
                type = 'fill',
                color = 'rgba(40,0,113,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')


In [33]:
red_counties_2016 = []
blue_counties_2016 = []

import re

for k, county in enumerate(county_names):
    county_cleaned = re.sub("\.", "", county)
    county_cleaned = re.sub(" ", "", county_cleaned)
    county_cleaned = re.sub(".*-", "", county_cleaned)
    row = fl_voting_data[fl_voting_data.County_Name == county_cleaned]
    if (len(row.index) != 1):
        print("Error: " + county_cleaned)
    else:
        if (row["votes_dem.16"].item() > row["votes_gop.16"].item()):
            blue_counties_2016.append(florida_data['features'][k])
        else:
            red_counties_2016.append(florida_data['features'][k])


NameError: name 'fl_voting_data' is not defined

In [34]:
red_data_2016 = {"type": "FeatureCollection"}
red_data_2016['features'] = red_counties_2016

blue_data_2016 = {"type": "FeatureCollection"}
blue_data_2016['features'] = blue_counties_2016

with open('florida-red-data-2016.json', 'w') as f:
    f.write(json.dumps(red_data_2016))
with open('florida-blue-data-2016.json', 'w') as f:
    f.write(json.dumps(blue_data_2016))


In [81]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = 'pk.eyJ1IjoiYWRhbWt1bGlkamlhbiIsImEiOiJjaXNya29kemwwNDNoMnRtMXBobGtvbWE4In0._k_JKvR8MknvKmiUsJ6C7g'

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
    )
])
layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/adnanhemani/data8-assets/master/florida-red-data-2016.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/adnanhemani/data8-assets/master/florida-blue-data-2016.json',
                type = 'fill',
                color = 'rgba(40,0,113,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')


In [7]:
df[df.State == "michigan"]

,County Name,State,ObamaVotes.12,RomneyVotes.12,votes_dem.16,votes_gop.16,total_votes.16,per_dem.16,per_gop.16,diff.16,...,public_transportation_to_work,working_at_home,occupation_management_business_science_arts,income_less_than_10000,income_10000_to_24999,income_25000_to_49999,income_50000_to_99999,income_100000_to_149999,income_150000_to_199999,income_more_than_200000
38,alcona,michigan,2462.0,3571.0,1732.0,4201.0,6179.0,0.280304,0.679883,1037.0,...,7.0,313.0,809.0,363.0,1229.0,1520.0,1194.0,202.0,202.0,52.0
44,alger,michigan,2212.0,2329.0,1663.0,2585.0,4513.0,0.368491,0.572790,2648.0,...,47.0,224.0,915.0,296.0,800.0,1194.0,1062.0,237.0,237.0,47.0
46,allegan,michigan,19594.0,29023.0,17932.0,33812.0,55153.0,0.325132,0.613058,515.0,...,92.0,1946.0,13907.0,2423.0,6495.0,12028.0,15327.0,4198.0,4198.0,762.0
58,alpena,michigan,6549.0,7298.0,4877.0,9090.0,14688.0,0.332040,0.618873,1674.0,...,41.0,665.0,3941.0,1119.0,3524.0,3876.0,3528.0,977.0,977.0,118.0
77,antrim,michigan,5107.0,7917.0,4448.0,8469.0,13572.0,0.327734,0.624005,1623.0,...,62.0,654.0,2706.0,669.0,2141.0,3043.0,3008.0,807.0,807.0,192.0
86,arenac,michigan,3669.0,4054.0,2238.0,4704.0,7457.0,0.300121,0.630817,1034.0,...,69.0,325.0,1654.0,570.0,1626.0,2241.0,1780.0,318.0,318.0,59.0
133,baraga,michigan,1574.0,1866.0,1156.0,2158.0,3486.0,0.331612,0.619048,2.0,...,27.0,81.0,1037.0,326.0,620.0,1098.0,1019.0,201.0,201.0,38.0
143,barry,michigan,11496.0,16651.0,9109.0,19197.0,30265.0,0.300975,0.634297,10.0,...,14.0,762.0,7014.0,955.0,3586.0,6372.0,8510.0,2409.0,2409.0,416.0
155,bay,michigan,26797.0,23735.0,21641.0,28327.0,52932.0,0.408845,0.535158,2233.0,...,283.0,1573.0,13927.0,3528.0,7931.0,13545.0,13709.0,4022.0,4022.0,634.0
193,benzie,michigan,4683.0,5071.0,4108.0,5539.0,10221.0,0.401918,0.541923,393.0,...,34.0,473.0,2441.0,372.0,1314.0,2510.0,2419.0,547.0,547.0,94.0


In [9]:
url = 'http://catalog.civicdashboards.com/dataset/08b4daa7-8888-4e03-ad73-ae2ff3c47771/resource/b12bbaa4-14df-4293-8636-2bb80804678c/download/89e296fe1c5c410d83c2d97155366e24temp.geojson'
michigan_data = requests.get(url)
michigan_data = michigan_data.json()

{'features': [{'geometry': {'coordinates': [[[[-85.864155, 46.702662],
       [-85.864932, 46.504757999999995],
       [-86.364953, 46.505852],
       [-86.36526099999999, 46.331928999999995],
       [-86.615397, 46.333262999999995],
       [-86.615601, 46.158691],
       [-87.11608, 46.158944999999996],
       [-87.11663399999999, 46.494808],
       [-87.112245, 46.749925999999995],
       [-87.112695, 47.817997999999996],
       [-86.94062199999999, 47.749905],
       [-86.949226, 47.749905],
       [-86.75012, 47.67196],
       [-86.75012, 47.674910999999994],
       [-86.631321, 47.624919999999996],
       [-86.634344, 47.624919999999996],
       [-86.625125, 47.617767],
       [-86.625125, 47.620008],
       [-86.50013, 47.568324],
       [-86.25013899999999, 47.477903],
       [-86.25013899999999, 47.474447999999995],
       [-86.125141, 47.423468],
       [-86.125142, 47.416744],
       [-86.020032, 47.374962],
       [-86.027345, 47.374961],
       [-86.00014499999999, 47.36188

In [11]:
county_names = []
county_names_dict = {}

for county in michigan_data['features']:
    for m in range(len(county['properties']['name'])):
        if county['properties']['name'][m:m+6] == 'County':
            county_names.append(county['properties']['name'][0:m-1].lower())
            county_names_dict[county['properties']['name'][0:m-1]] = county['properties']['name']
            
print(county_names)

['alger', 'cheboygan', 'barry', 'kalamazoo', 'menominee', 'grand traverse', 'mackinac', 'kalkaska', 'ottawa', 'roscommon', 'saginaw', 'ogemaw', 'tuscola', 'mason', 'ingham', 'ontonagon', 'branch', 'montmorency', 'osceola', 'wexford', 'alcona', 'allegan', 'huron', 'hillsdale', 'dickinson', 'chippewa', 'leelanau', 'clare', 'baraga', 'alpena', 'gratiot', 'wayne', 'oscoda', 'bay', 'muskegon', 'keweenaw', 'houghton', 'cass', 'crawford', 'newaygo', 'midland', 'gladwin', 'washtenaw', 'mecosta', 'luce', 'oceana', 'jackson', 'montcalm', 'sanilac', 'shiawassee', 'eaton', 'st. joseph', 'schoolcraft', 'st. clair', 'isabella', 'benzie', 'kent', 'emmet', 'monroe', 'gogebic', 'manistee', 'antrim', 'iosco', 'lake', 'livingston', 'delta', 'van buren', 'berrien', 'missaukee', 'macomb', 'iron', 'lenawee', 'ionia', 'arenac', 'calhoun', 'clinton', 'presque isle', 'charlevoix', 'otsego', 'genesee', 'lapeer', 'marquette', 'oakland']


In [12]:
red_counties_2012 = []
blue_counties_2012 = []
df = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/merged_election_data.csv").drop("Unnamed: 0", axis=1)
mi_voting_data = df[df.State == "michigan"]

In [22]:
import re

mi_voting_data.columns = [c.replace(' ', '_') for c in mi_voting_data.columns]

blue_counties_2016 = []
red_counties_2016 = []

for k, county in enumerate(county_names):
    county_cleaned = re.sub("\.", "", county)
    county_cleaned = re.sub(" ", "", county_cleaned)
    county_cleaned = re.sub(".*-", "", county_cleaned)
    row = mi_voting_data[mi_voting_data.County_Name == county_cleaned]
    if (len(row.index) != 1):
        print("Error: " + county_cleaned)
    else:
        if (row["votes_dem.16"].item() > row["votes_gop.16"].item()):
            blue_counties_2016.append(michigan_data['features'][k])
        else:
            red_counties_2016.append(michigan_data['features'][k])

    

In [23]:
red_data_2016 = {"type": "FeatureCollection"}
red_data_2016['features'] = red_counties_2016

blue_data_2016 = {"type": "FeatureCollection"}
blue_data_2016['features'] = blue_counties_2016

with open('florida-red-data-2016.json', 'w') as f:
    f.write(json.dumps(red_data_2016))
with open('florida-blue-data-2016.json', 'w') as f:
    f.write(json.dumps(blue_data_2016))


In [32]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = 'pk.eyJ1IjoiYWRuYW5oZW1hbmkiLCJhIjoiY2l5N3RnY3M4MDAxZTJxb3Y3anlsbnJ0ZSJ9.REcsaI8vU6ylkrpkjuMtqA'

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
    )
])
layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = red_counties_2016,
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = blue_counties_2016,
                type = 'fill',
                color = 'rgba(40,0,113,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')


In [31]:
red_counties_2016

[{'geometry': {'coordinates': [[[[-85.864155, 46.702662],
      [-85.864932, 46.504757999999995],
      [-86.364953, 46.505852],
      [-86.36526099999999, 46.331928999999995],
      [-86.615397, 46.333262999999995],
      [-86.615601, 46.158691],
      [-87.11608, 46.158944999999996],
      [-87.11663399999999, 46.494808],
      [-87.112245, 46.749925999999995],
      [-87.112695, 47.817997999999996],
      [-86.94062199999999, 47.749905],
      [-86.949226, 47.749905],
      [-86.75012, 47.67196],
      [-86.75012, 47.674910999999994],
      [-86.631321, 47.624919999999996],
      [-86.634344, 47.624919999999996],
      [-86.625125, 47.617767],
      [-86.625125, 47.620008],
      [-86.50013, 47.568324],
      [-86.25013899999999, 47.477903],
      [-86.25013899999999, 47.474447999999995],
      [-86.125141, 47.423468],
      [-86.125142, 47.416744],
      [-86.020032, 47.374962],
      [-86.027345, 47.374961],
      [-86.00014499999999, 47.361883],
      [-86.00014399999999, 47.3672